# Deep dive into RNNs

In [ ]:
import itertools
import matplotlib.pyplot as plt
import networkx as nx
import torch
from torch.nn import RNN
from torch.nn import LSTM
from torch.nn import GRU

In [ ]:
# https://networkx.org/documentation/stable/auto_examples/drawing/plot_multipartite_graph.html#


subset_sizes = [5, 5, 4, 3, 2, 4, 4, 3]
subset_color = [
    "gold",
    "violet",
    "violet",
    "violet",
    "violet",
    "limegreen",
    "limegreen",
    "darkorange",
]


def multilayered_graph(*subset_sizes):
    extents = nx.utils.pairwise(itertools.accumulate((0,) + subset_sizes))
    layers = [range(start, end) for start, end in extents]
    G = nx.Graph()
    for i, layer in enumerate(layers):
        G.add_nodes_from(layer, layer=i)
    for layer1, layer2 in nx.utils.pairwise(layers):
        G.add_edges_from(itertools.product(layer1, layer2))
    return G


G = multilayered_graph(*subset_sizes)
color = [subset_color[data["layer"]] for v, data in G.nodes(data=True)]
pos = nx.multipartite_layout(G, subset_key="layer")
plt.figure(figsize=(8, 8))
nx.draw(G, pos, node_color=color, with_labels=False)
plt.axis("equal")
plt.show()

In [ ]:
# import networkx as nx
# import itertools
# import matplotlib.pyplot as plt
# import numpy as np
# from matplotlib.patches import FancyArrowPatch

# # Taille des sous-ensembles (couches)
# subset_sizes = [3, 2, 3]

# # Couleurs pour les nœuds de chaque couche
# subset_color = [
#     "gold",
#     "violet",
#     "violet",
#     "violet",
#     "violet",
#     "limegreen",
#     "limegreen",
#     "darkorange",
# ]

# def multilayered_graph(*subset_sizes):
#     # Calcul des intervalles pour chaque couche
#     extents = nx.utils.pairwise(itertools.accumulate((0,) + subset_sizes))
#     layers = [range(start, end) for start, end in extents]
#     G = nx.DiGraph()  # Utilisation d'un graphe orienté pour représenter les boucles
#     for i, layer in enumerate(layers):
#         G.add_nodes_from(layer, layer=i)
#         node_list = list(layer)
#         # Ajout des arêtes entre tous les nœuds de la même couche
#         for u, v in itertools.permutations(node_list, 2):
#             G.add_edge(u, v)
#         # Ajout des boucles sur chaque nœud
#         for node in layer:
#             G.add_edge(node, node)
#     # Ajout des arêtes entre les couches adjacentes
#     for layer1, layer2 in nx.utils.pairwise(layers):
#         G.add_edges_from(itertools.product(layer1, layer2))
#     return G

# def draw_curved_edges(G, pos, ax):
#     # Création d'un dictionnaire pour stocker les arêtes entre les mêmes nœuds
#     edge_groups = {}
#     for u, v in G.edges():
#         if u == v:
#             # Boucle sur le même nœud
#             key = (u, v)
#         elif (v, u) in edge_groups:
#             # Arête existante dans l'autre sens
#             key = (v, u)
#         else:
#             key = (u, v)
#         edge_groups.setdefault(key, []).append((u, v))
    
#     for (u, v), edges in edge_groups.items():
#         num_edges = len(edges)
#         # Définition des courbures pour les arêtes multiples
#         if num_edges == 1:
#             rad_list = [0.0]
#         else:
#             rad_list = np.linspace(-0.5, 0.5, num_edges)
#         for (edge, rad) in zip(edges, rad_list):
#             u, v = edge
#             if u == v:
#                 # Boucle sur le même nœud avec une courbure fixe
#                 rad = 0.3
#             elif G.nodes[u]['layer'] == G.nodes[v]['layer']:
#                 # Ajustement de la courbure pour les arêtes dans la même couche
#                 rad *= 1.5
#             else:
#                 # Réduction de la courbure pour les arêtes entre couches
#                 rad *= 0.1
#             # Dessin de l'arête avec la courbure spécifiée
#             arrow = FancyArrowPatch(
#                 posA=pos[u], posB=pos[v],
#                 connectionstyle=f"arc3,rad={rad}",
#                 arrowstyle='-|>',
#                 mutation_scale=10.0,
#                 color='gray',
#                 linewidth=1.0,
#             )
#             ax.add_patch(arrow)

# G = multilayered_graph(*subset_sizes)
# color = [subset_color[data["layer"]] for v, data in G.nodes(data=True)]
# pos = nx.multipartite_layout(G, subset_key="layer")

# fig, ax = plt.subplots(figsize=(8, 8))
# nx.draw_networkx_nodes(G, pos, node_color=color, ax=ax)
# nx.draw_networkx_labels(G, pos, ax=ax)

# # Dessin des arêtes avec des courbes individuelles
# draw_curved_edges(G, pos, ax)

# plt.axis("equal")
# plt.axis('off')
# plt.show()


# RNN

In [ ]:
# model.state_dict()
# model.weight_ih_l*   e.g. model.weight_ih_l0
# model.weight_hh_l*   e.g. model.weight_hh_l0
# model.bias_ih_l*   e.g. model.bias_ih_l0
# model.bias_hh_l*   e.g. model.bias_hh_l0
#
# model.all_weights
# model.hidden_size
# model.input_size
#
# list(model.parameters())
# model.get_expected_hidden_size()
# model.get_parameter()
#
# print("model.weight_hh_l0:", model.weight_hh_l0)
# print("model.weight_ih_l0:", model.weight_ih_l0)
# print("model.bias_ih_l0:", model.bias_ih_l0)
# print("model.bias_hh_l0:", model.bias_hh_l0)

## One feature, one unit, one layer, no bias

<img src="assets/RNN1.drawio.svg" />

In [ ]:
model = RNN(input_size=1, hidden_size=1, num_layers=1, bias=False)
model.state_dict()

In [ ]:
x = torch.randn(10).unsqueeze(1)
x

In [ ]:
model(x)

## One feature, one unit, one layer, bias

<img src="assets/RNN2.drawio.svg" />

In [ ]:
model = RNN(input_size=1, hidden_size=1, num_layers=1, bias=True)
model.state_dict()

In [ ]:
x = torch.randn(10, 1, 1)    # (seq_len=10, batch_size=1, input_size=1)
x

In [ ]:
model(x)

## Three features, one unit, one layer, no bias

<img src="assets/RNN3.drawio.svg" />

In [ ]:
model = RNN(input_size=3, hidden_size=1, num_layers=1, bias=False)
model.state_dict()

In [ ]:
x = torch.randn(10, 3)
x

In [ ]:
model(x)

## Three features, one unit, one layer, bias

In [ ]:
model = RNN(input_size=3, hidden_size=1, num_layers=1, bias=False)
model.state_dict()

## One feature, two units, one layer, no bias

In [ ]:
model = RNN(input_size=1, hidden_size=2, num_layers=1, bias=False)
model.state_dict()

## One feature, two units, one layer, bias

In [ ]:
model = RNN(input_size=1, hidden_size=2, num_layers=1, bias=True)
model.state_dict()

## Three features, two units, one layer, no bias

<img src="assets/RNN4.drawio.svg" />

In [ ]:
model = RNN(input_size=3, hidden_size=2, num_layers=1, bias=False)
model.state_dict()

In [ ]:
x = torch.randn(10, 1, 3)    # (seq_len=10, batch_size=1, input_size=3)
x

In [ ]:
model(x)

## Three features, two units, one layer, bias

In [ ]:
model = RNN(input_size=2, hidden_size=2, num_layers=1, bias=False)
model.state_dict()

## Three features, three units, one layer, bias

In [ ]:
model = RNN(input_size=2, hidden_size=3, num_layers=1, bias=False)
model.state_dict()

## Three features, two units, two layers (stacked RNN), no bias

<img src="assets/RNN5.drawio.svg" />

In [ ]:
model = RNN(input_size=3, hidden_size=2, num_layers=2, bias=False)
model.state_dict()

In [ ]:
x = torch.randn(10, 1, 3)    # (seq_len=10, batch_size=1, input_size=3)
x

In [ ]:
y = model(x)
y

In [ ]:
type(y)

In [ ]:
type(y[0])

In [ ]:
type(y[1])

## Three features, two units, four layers (stacked RNN), no bias

<img src="assets/RNN6.drawio.svg" />

In [ ]:
model = RNN(input_size=3, hidden_size=2, num_layers=4, bias=False)
model.state_dict()

In [ ]:
x = torch.randn(10, 3)
x

In [ ]:
model(x)

# Bidirectionnal RNN

TODO...

# LSTM